# 1. Preparation
a. Install or import relevant libraries <br>
b. Runtime -> change runtime type -> choose any GPU type available for you <br>
c. Set variable `RUN_LOCALLY`: <br>
    - set it to True: you will run this notebook locally. <br>
    - set it to False: you will run this notebook on Colab. <br>

d. If you are running the notebook in Colab, you need to: <br>
    - specify the project path by setting the value of `project_path` <br>

In [ ]:

!pip install transformers
!pip install datasets
!pip install rouge_score
!pip install huggingface-hub

In [2]:
!nvidia-smi

Sat Apr 27 11:46:45 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   32C    P8              11W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

Set variable `RUN_LOCALLY`: <br>
    - set it to True: you will run this notebook locally. <br>
    - set it to False: you will run this notebook on Colab. <br>


### Running Locally: ###
Requirement: <br>
1. There are two files training.csv and eval.csv existing in folder `<project_folder>/data/dataset`.
2. There are processed PDF file in folder `<project_folder>/processed/`, such as `1901.00936v3.csv`.

### Running in Colab ###
Requirement: <br>
2. Specify the project path by setting the value of `project_path` in the below cell. <br>
1. training.csv, val.csv, and a pre-processed PDF file (e.g.`1901.00936v3.csv`) are put in the folder `data`. The folder `data` is located in the same Google Drive path as this notebook.

In [20]:
RUN_LOCALLY = True

import os, json, logging
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
import re

transformers_logger = logging.getLogger("transformers")
logging.getLogger("transformers").setLevel(logging.ERROR)
from transformers import LEDForConditionalGeneration, LEDTokenizer
from datasets import Dataset, load_dataset, load_metric, load_from_disk, DatasetDict
from rouge_score import rouge_scorer
pd.options.display.max_colwidth = 1000
torch.device('cpu')

######### set RUN_LOCALLY to True/False. Set project_path if the notebook is running in Colab ########
if RUN_LOCALLY:
    project_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
    # recently, the repo structure changes, and we need to follow this structure
    project_dataset_path = project_path + "/data/dataset"
    project_processed_data_path = project_path + "/processed"
    if not os.path.exists(project_dataset_path):
        raise Exception("The `dataset` folder does not exist.")
    if not os.path.exists(project_processed_data_path):
        raise Exception("The `processed` folder does not exist.")

else:
    # If you want to run on Colab and use GPU, only two extra data files are needed. You do not need to follow the above local data structure. Just ensure folder "data" exists in the same Google Drive folder as this notebook
    from google.colab import drive
    drive.mount("/content/drive")
    project_path = "/content/drive/MyDrive/Colab Notebooks/Course-NUS-cs5242-Neural Network and Deep Learning/final_project"

    project_dataset_path = project_path + "/data"
    project_processed_data_path = project_path + "/data"
    if not os.path.exists(project_dataset_path):
        raise Exception("The `data` folder does not exist.")
    


# 2. Define functions
This class `Model_operation` wraps all the functions related to the model's operation. At the end of the cell, we create an object of this class. This object will be used in the following trainning and testing steps.

In [10]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#Define the Model's operation in one class
class Model_operation:
    def __init__(self, model_name):
        self.model_name = model_name
        self.tokenizer = LEDTokenizer.from_pretrained(model_name, torch_dtype = torch.float16)
        self.model = LEDForConditionalGeneration.from_pretrained(model_name).to(DEVICE)
        self.config = LEDForConditionalGeneration.from_pretrained(model_name).config
        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=2e-5, weight_decay=0.01)

        self.rouge = load_metric("rouge")
        self.batchsize = 4
        self.max_input_length = 1024
        self.max_output_length = 128

    # tokenize the data
    def process_data_to_model_inputs(self, batch):
        inputs = self.tokenizer(
            batch['Text'],
            padding="max_length",
            truncation=True,
            max_length=self.max_input_length,
        )

        batch["input_ids"] = inputs.input_ids
        batch["attention_mask"] = inputs.attention_mask

        # put global attention on <s> token
        # according to https://github.com/huggingface/transformers/issues/18190, As you are running summarization, it is LEDForConditionalGeneration. For this model, we should put 1 for the global_attention_mask on the first token <s> in the encoder input sequence.
        batch["global_attention_mask"] = len(batch["input_ids"]) * [
            [1 if index == 0 else 0 for index in range(len(batch["input_ids"][0]))]
        ]

        outputs = self.tokenizer(
            batch['Groundtruth'],
            padding="max_length",
            truncation=True,
            max_length=self.max_output_length,
        )
        batch["labels"] = outputs.input_ids
        # We have to make sure that the PAD token is ignored by setting it to -100
        batch["labels"] = [
            [-100 if token == self.tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]
        return batch

    # Function to calculate ROUGE scores for generated summary and ground truth
    def calculate_rouge_scores(self, generated_summary, ground_truth_summary):
        scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

        rouge1_f1, rouge2_f1, rougeL_f1 = [], [], []
        for k in range(len(generated_summary)):
            scores = scorer.score(generated_summary[k], ground_truth_summary[k])
            rouge1_f1.append(scores['rouge1'].fmeasure)
            rouge2_f1.append(scores['rouge2'].fmeasure)
            rougeL_f1.append(scores['rougeL'].fmeasure)
        return np.mean(rouge1_f1), np.mean(rouge2_f1), np.mean(rougeL_f1)

    # convert the logits to real text
    def convert_logits_to_text(self, pred_logits):
        texts = []
        probs = torch.softmax(pred_logits, dim=-1)
        generated_ids = torch.argmax(probs, dim=-1)

        for i in range(len(generated_ids)):
            pred_str = self.tokenizer.decode(generated_ids[i], skip_special_tokens=True)
            texts.append(pred_str)
        # pred_str = self.tokenizer.decode(generated_ids, skip_special_tokens=True)
        # labels_ids[labels_ids == -100] = self.tokenizer.pad_token_id
        # label_str = self.tokenizer.batch_decode(labels_ids, skip_special_tokens=True)
        return texts

    # convert tokens to real text. The difference from convert_logits_to_text is that this function does not need to use softmax
    def convert_tokens_to_text(self, tokenized_sequences):
        texts = []
        for i in range(len(tokenized_sequences)):
          tokens_list = tokenized_sequences[i].tolist()
          if -100 in tokens_list:
              end_index = tokens_list.index(-100)
          else:
              end_index = len(tokens_list)
          pred_str = self.tokenizer.decode(tokenized_sequences[i][:end_index], skip_special_tokens=True)
          texts.append(pred_str)
        return texts

    def log_metrics(self,epoch, train_loss, val_loss, train_rouge_scores, val_rouge_scores):
        log_file = project_path + "/metrics_log.txt"
        with open(log_file, "a") as f:
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            log_str = f"{timestamp}, Epoch {epoch + 1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Train ROUGE: {train_rouge_scores}, Val ROUGE: {val_rouge_scores}\n"
            f.write(log_str)

    def log_generated_summary(self,epoch, expected_summary, generated_summary, other_info=""):
        log_file = project_path + "/generated_summary_log.txt"
        with open(log_file, "a") as f:
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            log_str = f"{timestamp}, Epoch {epoch + 1}, \nExpected: {expected_summary}, \nGenerated: {generated_summary}, \nOther Info: {other_info}\n"
            f.write(log_str)

    # tokenize the test data
    def process_test_data_to_model_inputs(self, batch):
        inputs = self.tokenizer(
            list(batch['text']),
            padding="max_length",    #  'do_not_pad'
            truncation=True,
            max_length=self.max_input_length,
        )

        batch["input_ids"] = inputs.input_ids
        batch["attention_mask"] = inputs.attention_mask

        # create 0 global_attention_mask lists
        batch["global_attention_mask"] = len(batch["input_ids"]) * [
            [1 if index ==0 else 0 for index in range(len(batch["input_ids"][0]))]
        ]
        return batch

    # test the model by passing it a pdf file dataframe. Return a dataframe with generated summary
    def generate_summary_for_user_pdf(self, pdf_df):
        data_df = pdf_df.copy()
        self.process_test_data_to_model_inputs(data_df)

        input_ids = torch.tensor(data_df["input_ids"]).to(DEVICE)
        test_am = torch.tensor(data_df["attention_mask"]).to(DEVICE)
        test_gam = torch.tensor(data_df["global_attention_mask"]).to(DEVICE)

        # when not passing labels, the outputs's loss will be None
        predicted_ids = self.model.generate(input_ids=input_ids,
                attention_mask = test_am, global_attention_mask = test_gam)

        generated = self.convert_tokens_to_text(predicted_ids)

        for idx, text in enumerate(generated):
            generated[idx] = re.sub(r"\\n", "", text).strip()

        pdf_df['generated'] = generated
        return pdf_df

    def show_model_state_dict(self):
        for key, value in self.model.state_dict().items():
            print(key, " - ", value.size())
        # for name, param in model_action.model.named_parameters():
        #     print(name, " - ", param.shape)

    def save_model_checkpoint(self, checkpoint_model_name):
        if not os.path.exists(project_path + "/Checkpoints"):
            os.mkdir(project_path + "/Checkpoints")
        checkpoint_path = project_path + "/Checkpoints/" +checkpoint_model_name
        torch.save(self.model.state_dict(), checkpoint_path)


model_action = Model_operation(model_name = "allenai/led-large-16384-arxiv")
# optional: set hyperparameters. Some parameters inherit from https://huggingface.co/docs/transformers/v4.40.1/en/main_classes/configuration#transformers.PretrainedConfig
model_action.config.num_beams = 2
model_action.config.max_length = model_action.max_output_length
model_action.config.min_length = 100
model_action.config.length_penalty = 2.0
model_action.config.early_stopping = True
model_action.config.no_repeat_ngram_size = 3
print("Pre-trained Model Config:\n", model_action.model.config)
print("Pre-trained Model state-dict:\n")
model_action.show_model_state_dict()

c:\Users\Claud\AppData\Local\Programs\Python\Python311\Lib\site-packages\datasets\load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Pre-trained Model Config:
 LEDConfig {
  "_name_or_path": "allenai/led-large-16384-arxiv",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "architectures": [
    "LEDForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "attention_window": [
    1024,
    1024,
    1024,
    1024,
    1024,
    1024,
    1024,
    1024,
    1024,
    1024,
    1024,
    1024
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
   

# 3. Prepare for DataSet and DataLoader
All the training data has been put into CSV file. We load CSV file and convert it to Huggingface Dataset. We then use Dataset's map method to tokenize the data. We format the Dataset and then load the data into PyTorch DataLoader to achieve the best performance.

The variable `CREATE_NEW_TOKENIZED_DATASET` decides if we use the existing tokenized dataset or re-create new ones. Using existing tokenized dataset saves time if the training data and verification data do not change from the previous training.

In [ ]:
CREATE_NEW_TOKENIZED_DATASET = True

train_df = pd.read_csv(project_dataset_path + "/training.csv")
val_df = pd.read_csv(project_dataset_path + "/eval.csv")
train_df.dropna(subset=['Text', 'Groundtruth'], inplace=True, axis=0)
val_df.dropna(subset=['Text', 'Groundtruth'], inplace=True, axis=0)

text_len = train_df['Text'].str.len()
plt.hist(text_len, bins = 20)
plt.title("Token Number Distribution")
plt.xlabel("Total Tokens")
plt.ylabel("Sample Number")
plt.show()

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

print("train_dataset records:", train_dataset.shape)
print("val_dataset records:", val_dataset.shape)
train_dataset = train_dataset.select(range(100))
val_dataset = val_dataset.select(range(40))
print("selected train_dataset records:", train_dataset.shape)
print("selected val_dataset records:", val_dataset.shape)

if CREATE_NEW_TOKENIZED_DATASET:
    # tokenize the data te prepare for the training
    train_dataset = train_dataset.map(
        model_action.process_data_to_model_inputs,
        batched=True,
        batch_size=model_action.batchsize)

    val_dataset = val_dataset.map(
        model_action.process_data_to_model_inputs,
        batched=True,
        batch_size=model_action.batchsize)

    train_dataset.set_format(
        type="torch",
        columns=["input_ids", "attention_mask", "global_attention_mask", "labels"],
    )
    val_dataset.set_format(
        type="torch",
        columns=["input_ids", "attention_mask", "global_attention_mask", "labels"],
    )
    # save to disk: https://huggingface.co/docs/datasets/v1.5.0/processing.html#:~:text=You%20can%20save%20your%20dataset,objects%2C%20you%20can%20use%20datasets.
    train_dataset.save_to_disk(project_dataset_path + "/train_dataset/")
    val_dataset.save_to_disk(project_dataset_path + "/val_dataset/")

else:
    # load from the existing formmated tokenized dataset
    train_dataset = load_from_disk(project_dataset_path + "/train_dataset/")
    val_dataset = load_from_disk(project_dataset_path + "/val_dataset")

train_data_loader = DataLoader(train_dataset, batch_size = model_action.batchsize, shuffle=True,drop_last=True)
val_data_loader = DataLoader(val_dataset, batch_size = model_action.batchsize, shuffle=True,drop_last=True)

In [ ]:
print(project_dataset_path)

train_df = pd.read_csv(project_dataset_path + "/training.csv")
train_df.head()
# train_df.dropna(subset=['Text', 'Groundtruth'], inplace=True, axis=0)
# train_df.head()

# 3. Fine-tune the Model and Save Check-point

The fine tune only occurs in the last layer: lm_head and final_logits_bias. All of the other layers' parameters are frozen.

There is early-exit mechanism. When the val_loss continues going up, stop the training process. The latest checkpoint model is saved to sub folder `Checkpoints`. All the training and verification loss, and rouge metrics are saved to folder `Runing_result`too.

In [6]:
num_epochs = 8
total_train_loss, total_val_loss, total_train_rouge, total_val_rouge = [], [], [], []
torch.backends.cuda.matmul.allow_fp16_reduced_precision_reduction = True
best_val_loss = float('inf')
#Declare variable for storing the checkpoint
checkpoint_filename = "LED_model_checkpoint.pt"
patience = 3

for epochi in tqdm(range(num_epochs)):
    val_batch_data_iter = iter(val_data_loader)
    train_loss, train_rouge = [], []
    val_loss, val_rouge = [], []
    for idx, data in tqdm(enumerate(train_data_loader)):
        ids = data['input_ids'].to(DEVICE)
        am = data['attention_mask'].to(DEVICE)
        gam = data['global_attention_mask'].to(DEVICE)
        labels = data['labels'].to(DEVICE)

        # freeze all the layers except the last layer lm_head
        model_action.model.train()
        for parameter in model_action.model.parameters():
            parameter.requires_grad = False
        for parameter in model_action.model.lm_head.parameters():
            parameter.requires_grad = True
        model_action.model.final_logits_bias.requires_grad = True

        # output = model_action.model(input_ids = ids, attention_mask = am, labels = labels, global_attention_mask = gam, use_cache = False)
        output = model_action.model(input_ids = ids, attention_mask = am, global_attention_mask = gam, labels = labels, use_cache = False)

        torch.device('cpu')
        loss = output.loss
        logits = output.logits

        print("epoch {}, batch {}, training loss: {}".format(epochi, idx, loss.item()))
        p_text = model_action.convert_logits_to_text(output.logits)[1:]
        e_text = model_action.convert_tokens_to_text(labels)
        model_action.log_generated_summary(epochi, e_text, p_text, f"training batch:{idx}")

        rouge1, rouge2, rougeL = model_action.calculate_rouge_scores(p_text, e_text)
        train_rouge.append((rouge1, rouge2, rougeL))

        # Backward and optimize
        model_action.optimizer.zero_grad()
        train_loss.append(loss.item())
        loss.backward()
        model_action.optimizer.step()

        # evaluate once every 20 mini-batches
        if idx > 0 and idx % 10 == 0:
            val_batch_data = next(iter(val_data_loader))

            # model. eval() will notify all your layers that you are in eval mode, that way, batchnorm or dropout layers will work in eval mode instead of training mode. torch. no_grad() impacts the autograd engine and deactivate it.
            model_action.model.eval()
            with torch.no_grad():
                val_data = next(val_batch_data_iter)
                val_ids = val_data['input_ids'].to(DEVICE)
                val_am = val_data['attention_mask'].to(DEVICE)
                val_gam = val_data['global_attention_mask'].to(DEVICE)
                val_labels = val_data['labels'].to(DEVICE)
                val_output = model_action.model(input_ids = val_ids, attention_mask = val_am, labels = val_labels, global_attention_mask = val_gam, use_cache = False)

            torch.device('cpu')
            vloss = val_output.loss.item()
            val_loss.append(vloss)

            print("epoch {}, prediction loss: {}".format(epochi, vloss))
            # The first word is always a repeat, so delete it.
            pred_text = model_action.convert_logits_to_text(val_output.logits)[1:]
            exp_text = model_action.convert_tokens_to_text(val_labels)
            model_action.log_generated_summary(epochi, e_text, p_text, f"verification batch:{idx}")
            rouge1, rouge2, rougeL = model_action.calculate_rouge_scores(pred_text, exp_text)
            val_rouge.append((rouge1, rouge2, rougeL))
            print("\nprediction text:")
            display(pred_text)

    total_train_loss.append(train_loss)
    total_val_loss.append(val_loss)
    avg_train_loss = np.mean(train_loss)
    avg_val_loss = np.mean(val_loss)
    total_train_rouge.append(train_rouge)
    total_val_rouge.append(val_rouge)
    model_action.log_metrics(epochi, avg_train_loss, avg_val_loss, train_rouge, val_rouge)
    print("epoch {}'s average training loss: {}", np.mean(train_loss))
    print("epoch {}'s average verification loss: {}", np.mean(val_loss))

    # The following uses Sudha's code to stop training and save checkpoint when the val loss improves
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        epochs_no_improve = 0
        model_action.save_model_checkpoint(checkpoint_filename)
        print("The checkpoint model is saved after finishing epoch {epochi}")
    else:
        epochs_no_improve += 1
        if epochs_no_improve == patience:
            print(f"Early stopping triggered.")
            break

# save running result for evaluation
running_res_folder = project_path + "/Running_result"
if not os.path.exists(running_res_folder):
    os.mkdir(running_res_folder)

tt_loss = pd.DataFrame(total_train_loss)
tt_loss.to_csv(running_res_folder + "/total_train_loss.csv")
tv_loss = pd.DataFrame(total_val_loss)
tt_loss.to_csv(running_res_folder + "/total_val_loss.csv")

tt_train_rouge = pd.DataFrame(total_train_rouge)
tt_train_rouge.to_csv(running_res_folder + "/total_train_rouge.csv")
tt_val_rouge = pd.DataFrame(total_val_rouge)
tt_val_rouge.to_csv(running_res_folder + "/total_val_rouge.csv")


  0%|          | 0/8 [00:00<?, ?it/s]
0it [00:00, ?it/s]

epoch 0, batch 0, training loss: 3.61143159866333



1it [00:05,  5.12s/it]

epoch 0, batch 1, training loss: 3.643406867980957



2it [00:08,  4.06s/it]

...

epoch 7, batch 23, training loss: 3.97924542427063



24it [01:26,  3.60s/it]

epoch 7, batch 24, training loss: 3.7098803520202637



25it [01:29,  3.60s/it]
100%|██████████| 8/8 [12:49<00:00, 96.22s/it]


epoch {}'s average training loss: {} 3.3767225456237795
epoch {}'s average verification loss: {} 3.9122928380966187


# 4. Inference: Test PDF Summarization
Load a CSV file from `processed` folder. This CSV file is created after running `processing_pdf.jpynb`. The text of sections, subsections, subsubsections are put in different rows and marked with section number and section titles.

Use your own CSV file name to replace the sample `1901.00936v3.csv`.

In [7]:
pdf_df = pd.read_csv(project_processed_data_path + "/1901.00936v3.csv")

print("Start generating summary...")
generated_df = model_action.generate_summary_for_user_pdf(pdf_df)
display(generated_df.head())


Start generating summary...


,text,section_num:,section,subsection,generated
0,"an efficient linux kernel implementation of service function chaining for legacy vnfs based on ipv6 segment routing andrea mayer*, stefano salsano*, pier luigi ventre*, ahmed abdelsalam {double dagger} {section sign}, luca chiaraviglio*, clarence filsfils {section sign} *university of rome tor vergata, cnit, {double dagger}gran sasso science institute, {section sign}cisco systems extended version of the paper accepted to ieee netsoft 2019 - v04 - july 2019 abstract {em dash}we consider the ipv6 segment routing technology for service function chaining of virtual network functions . most of the vnfs are legacy vnfs and expect to process traditional ip packets. an sr proxy is needed to support them. we have extended the implementation of srv6 in the linux kernel, realizing an open source sr-proxy, referred to as srnk . the performance of the proposed solution has been evaluated, identifying a poor scalability with respect to the number of vnfs to be supported in a node. therefore we p...",Abstract,Abstract,NaN,"we consider the ipv6 segment routing technology for service function chaining of virtual network functions ( vnfs ). \n most of the vnfs are legacy vnfs and expect to process traditional ip packets. \n an sr proxy is needed to support them. we have extended the implementation of srv6 in the linux kernel, realizing an open source sr-proxy, referred to as srnk. \n the performance of the proposed solution has been evaluated, identifying a poor scalability with respect to the number of vnfs to be supported in a node. therefore we provided a second design, enhancing the linux policy routing framework. \n the performance of srnkv2 is independent from the number of supported vnfs in a node. we compared the performance of srnkv2 with a reference scenario not performing the encapsulation and decapsulation operation and demonstrated that the overhead of srnkv2 is very small, on the order of 3.5%."
1,"network operators are facing difficult challenges to keep up with the increasing demand for capacity, the need to support fast service creation and at the same time the goal of reducing the costs. network function virtualization and software defined networking represent an answer to these challenges and are changing the way ip networks are designed and operated. leveraging cloud computing principles, nfv moves the traditional data-plane network functions from expensive, closed and proprietary hardware to the so-called virtual network functions running over a distributed, cloud-like infrastructure referred to as nfvi . the sdn architecture splits the data and control planes and moves the intelligence to the sdn controller. sdn aims at simplifying the introduction of new services and fostering flexibility thanks to the centralized network state view. the concept of services chaining is directly associated to nfv. actually, the idea of creating a processing path across services pre-da...",I,I. INTRODUCTION,NaN,"the ietf sfc working group has investigated the scenarios and issues related to dynamic service chaining and proposed a reference architecture. \n the sfc framework proposed in does not pose any restriction on the function that can be chained: they can be both virtualized or physical functions. for the sake of simplicity, \n we will only refer to the virtualized case and will simply use the term vnf instead of service function. the ietf sfc wg is considering the network service header as a specific solution for the realization of the sfc architecture. the nsh header defines the service-level data-plane encapsulation for realizing the vnfs chaining. moreover, it defines the packet meta-data that arxiv:1901.00936v3 23 jul 2019 can be inserted into the header to exchange state between the nodes of the sfc architecture. in this work we are advocating the use of ipv6 segment routing to implement service function chaining,. segment routing, is a form of source routing, which allows to ad.